In [1]:
conda install pytz

Retrieving notices: ...working... done
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [ ]:

import boto3
import datetime
import pytz


# Create EC2 client
ec2 = boto3.client('ec2',region_name='us-east-1a')

# Get all EC2 instances with Name tag 'dpt-web-server'
instances = ec2.describe_instances(Filters=[
    {'Name': 'tag:Name', 'Values': ['dpt-web-server']}
])

# Get the instance IDs as a list
instance_ids = []
for reservation in instances['Reservations']:
    for instance in reservation['Instances']:
        instance_ids.append(instance['InstanceId'])

# Create AMIs for each instance
for instance_id in instance_ids:
    try:
        # Get the Name tag of the instance
        name_tag = ""
        for tag in ec2.describe_tags(Filters=[{'Name': 'resource-id', 'Values': [instance_id]}, {'Name': 'key', 'Values': ['Name']}])['Tags']:
            if tag['Key'] == 'Name':
                name_tag = tag['Value']
        
        # Create an AMI for the instance
        image_name = name_tag + "-" + datetime.datetime.now(pytz.utc).strftime("%Y-%m-%d")
        response = ec2.create_image(
            Description='AMI pour la sauvegarde hebdomadaire',
            InstanceId=instance_id,
            Name=image_name,
            NoReboot=True,
            DryRun=False
        )
        
        # Add tags to the AMI
        ami_id = response['ImageId']
        ec2.create_tags(Resources=[ami_id], Tags=[{'Key': 'Name', 'Value': name_tag}])
        ec2.create_tags(Resources=[ami_id], Tags=[{'Key': 'Description', 'Value': 'AMI pour la sauvegarde hebdomadaire'}])
        
        print("AMI " + ami_id + " créée pour l'instance " + instance_id)
    except Exception as e:
        print("Exception lors de la création de l'AMI pour l'instance " + instance_id + " : " + str(e))

# Delete unused AMIs older than 30 days
images = ec2.describe_images(Filters=[
    {'Name': 'tag:Description', 'Values': ['AMI pour la sauvegarde hebdomadaire']},
    {'Name': 'tag:Name', 'Values': ['dpt-web-server*']}
])
for image in images['Images']:
    try:
        create_date = image['CreationDate']
        image_age = datetime.datetime.now(pytz.utc) - datetime.datetime.strptime(create_date, '%Y-%m-%dT%H:%M:%S.%fZ').replace(tzinfo=pytz.utc)
        if image_age.days > 30:
            ec2.deregister_image(ImageId=image['ImageId'])
            print("AMI " + image['ImageId'] + " supprimée.")
    except Exception as e:
        print("Exception lors de la suppression de l'AMI " + image['ImageId'] + " : " + str(e))
